# Check pnl across trades and positions

* Compare trade pnl between two subgraphs
* Compare position pnl between two subgraphs


In [84]:
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import json
import sqlite3
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.io as pio
import plotly.express as px
from web3 import Web3
from web3.middleware import geth_poa_middleware

load_dotenv()

pio.renderers.default = 'notebook'


In [85]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# set up sqlite connection
con = sqlite3.connect("../../data/perps.db")

# get a web3 provider
w3 = Web3(Web3.HTTPProvider(RPC_ENDPOINT))


In [86]:
# data contract
with open('../abi/PerpsV2Data.json', 'r') as file:
    PerpsV2DataAbi = json.dumps(json.load(file))

PerpsV2DataAddress = "0xF7D3D05cCeEEcC9d77864Da3DdE67Ce9a0215A9D"


## Get transfers and trades

In [87]:
df_trade = pd.read_sql_query('SELECT * FROM trades', con)
df_transfer = pd.read_sql_query('SELECT * FROM transfers', con)


In [88]:
## summarize transfers
df_transfer_summ = df_transfer.groupby('asset')['size'].sum().reset_index()
df_transfer_summ.columns = ['asset', 'netTransfers']

## summarize trades
df_trade_summ = df_trade.groupby('asset')['feesPaid'].sum().reset_index()
df_trade_summ.columns = ['asset', 'feesPaid']


In [89]:
df_trade_summ

,asset,feesPaid
0,AAVE,5094.591762
1,APE,30830.140721
2,ATOM,28423.087364
3,AUD,84.457861
4,AVAX,43037.064396
5,AXS,2678.255616
6,BNB,50993.312869
7,DOGE,10508.201371
8,DYDX,35698.544667
9,EUR,2142.781228


## Get the market data

In [90]:
perpsV2Data = w3.eth.contract(address=PerpsV2DataAddress, abi=PerpsV2DataAbi)
marketSummaries = perpsV2Data.functions.allProxiedMarketSummaries().call()



In [91]:
markets = [{
    'asset': market[1].decode().replace('\x00', ''),
    'marketDebt': w3.fromWei(market[7], unit='ether')
} for market in marketSummaries]

df_markets = pd.DataFrame(markets)
df_markets['marketDebt'] = df_markets['marketDebt'].astype(float)
df_markets

,asset,marketDebt
0,sETH,8.079723e+06
1,sBTC,8.345421e+06
2,LINK,3.864533e+05
3,SOL,6.703912e+05
4,AVAX,2.814628e+05
5,AAVE,2.275315e+03
6,UNI,1.013287e+05
7,MATIC,4.584861e+05
8,APE,6.486755e+04
9,DYDX,2.878955e+04


In [93]:
df_pnl = df_markets.merge(df_transfer_summ, on='asset').merge(df_trade_summ, on='asset')
df_pnl['netPnl'] = df_pnl['marketDebt'] - df_pnl['netTransfers'] - df_pnl['feesPaid']
df_pnl

,asset,marketDebt,netTransfers,feesPaid,netPnl
0,sETH,8.079723e+06,8.839827e+06,937474.306662,-1.697578e+06
1,sBTC,8.345421e+06,8.603859e+06,796713.946812,-1.055152e+06
2,LINK,3.864533e+05,4.669901e+05,93261.869690,-1.737986e+05
3,SOL,6.703912e+05,7.336359e+05,111749.912617,-1.749947e+05
4,AVAX,2.814628e+05,2.109170e+05,43037.064396,2.750878e+04
5,AAVE,2.275315e+03,3.721074e+02,5094.591762,-3.191384e+03
6,UNI,1.013287e+05,1.152431e+05,10702.703136,-2.461703e+04
7,MATIC,4.584861e+05,5.459265e+05,87429.915699,-1.748703e+05
8,APE,6.486755e+04,3.509849e+04,30830.140721,-1.061084e+03
9,DYDX,2.878955e+04,4.885604e+04,35698.544667,-5.576504e+04


In [94]:
df_pnl.to_csv('output/market_upnl.csv', index=False)

In [95]:
df_pnl['netPnl'].sum()

-3870885.193853333

In [96]:
df_pnl['netPnlClean'] = df_pnl['netPnl'].apply(lambda x: round(x, 2))
df_pnl[['asset', 'netPnlClean']].sort_values('netPnlClean', ascending=True)

,asset,netPnlClean
0,sETH,-1697577.98
1,sBTC,-1055151.63
11,OP,-293278.34
3,SOL,-174994.69
7,MATIC,-174870.30
2,LINK,-173798.61
19,FTM,-127390.52
9,DYDX,-55765.04
16,ATOM,-46561.79
10,BNB,-26936.16
